In [1]:
import openai
import re
import string
import numpy as np
from tqdm import tqdm
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

In [74]:


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

def call_instructgpt_with_answers(questions, model_predictions, ground_truths):
    # if exact_match_score(model_prediction, ground_truth):
    #     return 1
    api_key = open("/data/katie_kang/openai_key_file.txt", "r").read()
    openai.api_key = api_key.strip()

    prompt_template1 = """Are the answers equivalent?

Ex 1:
Q: Who was leader of Revolutionary Army?
A1: George Washington
A2: US President Washington

Ex 2:
Q: Who is the screenwriter of Dead Silence?
A1: James Wan
A2: Leigh Whannell

Ex 3:
Q: country of citizenship of McKean?
A1: United States
A2: American

Ex 4:
Q: What genre is Taxi for Two?
A1: romantic comedy film
A2: comedy

""" 


    prompt_template2 = None

    for i in range(len(questions)):
        if i == 0:
            prompt_template2 = """Ex {}:
Q: {}
A1: {}
A2: {}

""".format(i+5, questions[i], model_predictions[i], ground_truths[i])
        else:
            prompt_template2 += """Ex {}:
Q: {}
A1: {}
A2: {}

""".format(i+5, questions[i], model_predictions[i], ground_truths[i])

    prompt_template3 = """Ex 1 Equivalent? Yes
Ex 2 Equivalent? No
Ex 3 Equivalent? Yes
Ex 4 Equivalent? Yes
"""

    prompt = prompt_template1 + prompt_template2 + prompt_template3

    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",  # or another model version
        # prompt=[filled_prompt, filled_prompt],
        prompt=prompt,
        max_tokens=200,
        temperature=0.0,
    )

    response = response.choices[0].text.strip()

    try:
        split_response = response.split("\n")
        split_response = [split_response_i.split("? ")[1] for split_response_i in split_response]
        linguistically_equivalent = [split_response_i=="Yes" for split_response_i in split_response]
        assert(len(linguistically_equivalent) == len(questions))
    except:
        print(response)
        linguistically_equivalent = [False for _ in range(len(questions))]


    return np.array(linguistically_equivalent)
    

In [45]:
data = np.load('ckpts/sft_ctrex_llama7B_2_commit_lr1e-5_2/checkpoint_30000/hf_model/output_strings_train.npy')


In [30]:

questions_NEM = []
answers_NEM = []
predictions_NEM = []
exact_match_idxs = []
not_exact_match_idxs = []
for j in tqdm(range(len(data))):
    line = data[j]
    answer = line.split('Label: ')[1].strip()
    try:
        prediction = line.split('The answer is')[1].split('Label:')[0].strip()[0:-1]
    except:
        prediction = line.split('The Answer Is')[1].split('Label:')[0].strip()[0:-1]
    question = line.split('<unk>')[-1].split('?')[0].strip() + '?'

    if exact_match_score(answer, prediction):
        exact_match_idxs.append(j)
    else:
        questions_NEM.append(question)
        answers_NEM.append(answer)
        predictions_NEM.append(prediction)
        not_exact_match_idxs.append(j)



100%|██████████| 581000/581000 [00:06<00:00, 96788.95it/s]


In [31]:
questions_NEM = np.array(questions_NEM)
answers_NEM = np.array(answers_NEM)
predictions_NEM = np.array(predictions_NEM)

In [75]:
linguistically_equivalent_idxs = []
not_linguistically_equivalent_idxs = []

num_points_per_prompt = 20
# for i in tqdm(range(0, len(not_exact_match_idxs)//num_points_per_prompt)):
for i in tqdm(range(10)):
    results = call_instructgpt_with_answers(questions_NEM[i*num_points_per_prompt:(i+1)*num_points_per_prompt], 
    answers_NEM[i*num_points_per_prompt:(i+1)*num_points_per_prompt]
    , predictions_NEM[i*num_points_per_prompt:(i+1)*num_points_per_prompt])

    points_in_consideration = np.arange(i*num_points_per_prompt, (i+1)*num_points_per_prompt)
    linguistically_equivalent_idxs.append(points_in_consideration[np.where(results == 1)[0]])
    not_linguistically_equivalent_idxs.append(points_in_consideration[np.where(results == 0)[0]])

    if i%2 == True:
        equivalence = np.ones(len(data))*-1
        equivalence[np.array(exact_match_idxs)] = 1
        equivalence[np.concatenate(linguistically_equivalent_idxs)] = 1
        equivalence[np.concatenate(not_linguistically_equivalent_idxs)] = 0
        np.save(equivalence, "ckpts/sft_ctrex_llama7B_2_commit_lr1e-5_2/checkpoint_30000/hf_model/output_strings_train_linguistic_equivalence.npy")

  0%|          | 0/10 [00:00<?, ?it/s]

2154


ZeroDivisionError: division by zero